# Getting Started with Landsat

This notebook is a very simple example of the fundamentals of working with
Earth observation data using cloud native geospatial tools and practices.

## Import libraries

In [10]:
from pystac_client import Client
from odc.stac import load, configure_s3_access

## Configure the environment

In [11]:
# STAC Catalog URL
catalog = "https://earth-search.aws.element84.com/v1"

# Create a STAC Client
client = Client.open(catalog)

# This line will fail if you don't have credentials configured
_ = configure_s3_access(cloud_defaults=True, requester_pays=True)

## Pick a study area

In [12]:
# Find a location you're interested in on Google Maps and copy the coordinates
# by right-clicking on the map and clicking the coordinates

# These coords are in the order Y then X, or Latitude then Longitude
coords = 9.9251702,118.7817471,13.79  # Honda Bay, Palawan, Philippines
buffer = 0.2
bbox = (coords[1] - buffer, coords[0] - buffer, coords[1] + buffer, coords[0] + buffer)
landsat_stretch = dict(vmin=7500, vmax=18000)

datetime = "2024-06"

## Find STAC Items

A STAC Item is a record of a "scene" or "granule". It has a spatial and temporal
extent, a set of Assets, or links to files, and important metadata that can
be used to filter scenes, for example, for percentage cloud cover for the scene.

In the example below, we're searching the Landsat Collection 2, level 2 STAC Collection
for items in the bounding box and datetime range with cloud cover less than 50%.

In [ ]:
items = client.search(
    collections=["landsat-c2-l2"],
    bbox=bbox,
    datetime=datetime,
    query={"eo:cloud_cover": {"lt": 50}},
).item_collection()

print(f"Found {len(items)} items")

## Load data

This uses the Python library `odc-stac` to handle loading of the actual data
from the STAC Items. The `chunks` argument instructs the tool to use Dask
to lazy-load the data.

Note that while we found four STAC Items above, we will have two timesteps
in the Xarray dataset below, because some scenes are captured on the same day
and are adjacent, so they've been merged using the `groupby="solar_day"` argument.

In [ ]:
data = load(
    items,
    bbox=bbox,
    collection="landsat-c2-l2",
    measurements=["red", "green", "blue"],
    groupby="solar_day",
    chunks={"x": 2048, "y": 2048},
)
data

In [15]:
from odc.stac import configure_rio

configure_rio(cloud_defaults=True, GDAL_HTTP_PROXY=None)

## Visualise data

This step uses `matplotlib` to view data as a static image. It takes a longer time to
run than previous steps, because it's actually loading the data to prepare the images.

The `to_array()` function is a trick used to be able to visualise the data as a
red, green, blue "true colour" image.

In [ ]:
data.to_array().plot.imshow(col="time", col_wrap=2, size=6, **landsat_stretch)

### Interactive map

This step uses another `odc` tool to visualise the data on a map.

In [ ]:
best = data.sel(time="2024-06-11").squeeze()

# The odc.explore function will choose red, green and blue by default
# but you can choose bands with the bands=["b1", "b2", "b3"] argument
best.odc.explore(**landsat_stretch)

## Export data

Here we write data to disk, again using an `odc` tool.

In [ ]:
best.odc.to_rgba(**landsat_stretch).odc.write_cog("landsat_example.tif", overwrite=True)

## Summary

In this notebook we've learnt how to search for Landsat scenes using a STAC API
and then to load and visualise that data, using `odc-stac`.

In future notebooks, we'll be using this basic functionality to find and load
data from Landsat, Sentinel-2 and Sentinel-1 and to run more complex analyses.